![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)
# Routing Optimization

Implementing a semantic router is a great light weight way to add branching logic to your application without taking on additional LLM calls. However, it can be tough to determine the optimal distance threshold values for your routes to maximize performance. This guide will walk through:

- how to configure a semantic router
- how to optimize the distance thresholds for the routes
- a comparison between performing similar logic with an LLM versus a router

## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/semantic-router/01_routing_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Setup

## Install Packages

In [ ]:
%pip install -q sentence-transformers ranx

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# install from branch since scheduled for 0.5.0
%pip install git+https://github.com/redis/redis-vl-python.git@feat/RAAE-602/threshold-optimizer

  Cloning https://github.com/redis/redis-vl-python.git (to revision feat/RAAE-602/threshold-optimizer) to /private/var/folders/_g/rr4lnxxx1_z7m78lz89dhvsm0000gp/T/pip-req-build-ykzynneq
  Running command git clone --filter=blob:none --quiet https://github.com/redis/redis-vl-python.git /private/var/folders/_g/rr4lnxxx1_z7m78lz89dhvsm0000gp/T/pip-req-build-ykzynneq
  Running command git checkout -b feat/RAAE-602/threshold-optimizer --track origin/feat/RAAE-602/threshold-optimizer
  Switched to a new branch 'feat/RAAE-602/threshold-optimizer'
  branch 'feat/RAAE-602/threshold-optimizer' set up to track 'origin/feat/RAAE-602/threshold-optimizer'.
  Resolved https://github.com/redis/redis-vl-python.git to commit 18ff1008c5a40353c97c176d3d30028a87ff777a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached ml_dtypes-0.4.1-cp311-cp311-macosx_10_9_universal2.whl.metadata (20 kB)
  Using cached pytho

## Run a Redis instance

#### For Colab
Use the shell script below to download, extract, and install [Redis Stack](https://redis.io/docs/getting-started/install-stack/) directly from the Redis package archive.

In [ ]:
# NBVAL_SKIP
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

#### For Alternative Environments
There are many ways to get the necessary redis-stack instance running
1. On cloud, deploy a [FREE instance of Redis in the cloud](https://redis.com/try-free/). Or, if you have your
own version of Redis Enterprise running, that works too!
2. Per OS, [see the docs](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/)
3. With docker: `docker run -d --name redis-stack-server -p 6379:6379 redis/redis-stack-server:latest`

### Define the Redis Connection URL

By default this notebook connects to the local instance of Redis Stack. **If you have your own Redis Enterprise instance** - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [17]:
import os
import warnings

warnings.filterwarnings("ignore")

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost") # ex: "redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "6379")      # ex: 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  # ex: "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

# Routing with multiple routes

## Define the Routes

Below we define 3 different routes. One for `technology`, one for `sports`, and
another for `entertainment`. Now for this example, the goal here is
surely topic "classification". But you can create routes and references for
almost anything.

Each route has a set of references that cover the "semantic surface area" of the
route. The incoming query from a user needs to be semantically similar to one or
more of the references in order to "match" on the route. Note that each route can have it's own distinct `distance_threshold` that defines what is considered a match for the particular query. 

In [5]:
from redisvl.extensions.router import Route

# Define routes for the semantic router
technology = Route(
    name="technology",
    references=[
        "what are the latest advancements in AI?",
        "tell me about the newest gadgets",
        "what's trending in tech?"
    ],
    metadata={"category": "tech", "priority": 1},
    distance_threshold=0.5
)

sports = Route(
    name="sports",
    references=[
        "who won the game last night?",
        "tell me about the upcoming sports events",
        "what's the latest in the world of sports?",
        "sports",
        "basketball and football"
    ],
    metadata={"category": "sports", "priority": 2},
    distance_threshold=0.7
)

entertainment = Route(
    name="entertainment",
    references=[
        "what are the top movies right now?",
        "who won the best actor award?",
        "what's new in the entertainment industry?"
    ],
    metadata={"category": "entertainment", "priority": 3},
    distance_threshold=0.6
)

## Initialize the SemanticRouter

Like before the ``SemanticRouter`` class will automatically create an index within Redis upon initialization for the route references.

In [7]:
import os
from redisvl.extensions.router import SemanticRouter

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Initialize the SemanticRouter
multi_topic_router = SemanticRouter(
    name="topic-router",
    routes=[technology, sports, entertainment],
    redis_url="redis://localhost:6379",
    overwrite=True # Blow away any other routing index with this name
)

/Users/robert.shelton/.pyenv/versions/3.11.8/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/robert.shelton/.pyenv/versions/3.11.8/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## View the created index

In [8]:
# look at the index specification created for the semantic router
!rvl index info -i topic-router



Index Information:
╭──────────────┬────────────────┬──────────────────┬─────────────────┬────────────╮
│ Index Name   │ Storage Type   │ Prefixes         │ Index Options   │   Indexing │
├──────────────┼────────────────┼──────────────────┼─────────────────┼────────────┤
│ topic-router │ HASH           │ ['topic-router'] │ []              │          0 │
╰──────────────┴────────────────┴──────────────────┴─────────────────┴────────────╯
Index Fields:
╭────────────┬─────────────┬────────┬────────────────┬────────────────┬────────────────┬────────────────┬────────────────┬────────────────┬─────────────────┬────────────────╮
│ Name       │ Attribute   │ Type   │ Field Option   │ Option Value   │ Field Option   │ Option Value   │ Field Option   │   Option Value │ Field Option    │ Option Value   │
├────────────┼─────────────┼────────┼────────────────┼────────────────┼────────────────┼────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┤
│ route_name │ route

## Test it out

In [9]:
# Query the router with a statement
route_match = multi_topic_router("Can you tell me about the latest in artificial intelligence?")
route_match

RouteMatch(name='technology', distance=0.419145862261)

## Optimize route distance thresholds with test data

For optimization within redisvl you can create test data manually or make use of a model to generate some for you. In this case we will use a model to do it for us.

Prompt for creating test data:
> used claude sonnet 3.7 for generation of resource

```txt
You are a test data creation helper. 

Create test data of the form:

{
    "query": "query about a topic",
    "query_match": "topic-the-query-matches"
}

The 3 available topics are: technology, sports, and entertainment. Generate many examples that map to these topics such that we can train a model to find the best thresholds for this classification task. Also make sure to include some examples that don't map to any of the topics to check the null case for these leave the query_match field empty.
```

The output of this call was saved to `./resources/test_data.json`

In [ ]:
import json

with open("resources/train_data.json", "r") as f:
    train_data = json.load(f)

## Run optimization with router

In [ ]:
from redisvl.utils.optimize import RouterThresholdOptimizer

optimizer = RouterThresholdOptimizer(multi_topic_router, train_data)
optimizer.optimize()

/Users/robert.shelton/.pyenv/versions/3.11.8/lib/python3.11/site-packages/ranx/metrics/f1.py:36: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  scores[i] = _f1(qrels[i], run[i], k, rel_lvl)


Eval metric F1: start 0.617, end 0.667 
Ending thresholds: {'technology': 0.7242424242424242, 'sports': 0.4696969696969694, 'entertainment': 0.4666666666666667}


## Test classification against LLM

Using the same prompt above we generated and stored another 20 questions to use as our `test_data` to compare against using an LLM model to perform this classification.

In [14]:
import os
import getpass
import time
import numpy as np

from openai import OpenAI

os.environ["TOKENIZERS_PARALLELISM"] = "False"

api_key = os.getenv("OPENAI_API_KEY") or getpass.getpass("Enter your OpenAI API key: ")

client = OpenAI(api_key=api_key)

def ask_openai(question: str) -> str:
    prompt = f"""
    You are a classification bot. Your job is to classify the following query as either technology, sports, entertainment, or none. Return only the string label or an empty string if no match.

    query: "{question}"
    """
    response = client.completions.create(
      model="gpt-3.5-turbo-instruct",
      prompt=prompt,
      max_tokens=200
    )
    return response.choices[0].text.strip()

In [ ]:
with open("resources/test_data.json", "r") as f:
    test_data = json.load(f)


ask_openai(test_data[0]["query"])

'technology'

In [32]:
import time

def test_classifier(classifier, test_data, is_router=False):
    correct = 0
    times = []
    for data in test_data:
        start = time.time()
        if is_router:
            prediction = classifier(data["query"]).name
        else:
            prediction = classifier(data["query"])
        
        if not prediction or prediction.lower() == "none":
            prediction = ""

        times.append(time.time() - start)
        print(f"Expected | Observed: {data['query_match']} | {prediction.lower()}")
        if prediction.lower() == data["query_match"]:
            correct += 1
    accuracy = correct / len(test_data)
    avg_time = np.mean(times)
    return accuracy, avg_time

In [33]:
llm_accuracy, llm_avg_time = test_classifier(ask_openai, test_data)

Expected | Observed: technology | the string label for this query is technology, since it is discussing various areas of artificial intelligence, which classify as technology topics.
Expected | Observed: technology | result: technology
Expected | Observed: technology | technology
Expected | Observed: technology | technology
Expected | Observed: technology | technology
Expected | Observed: sports | sports
Expected | Observed: sports | sports
Expected | Observed: sports | 
Expected | Observed: sports | sports
Expected | Observed: sports | the label for this query is "sports".
Expected | Observed: entertainment | entertainment
Expected | Observed: entertainment | entertainment
Expected | Observed: entertainment | possible match: entertainment
Expected | Observed: entertainment | entertainment
Expected | Observed: entertainment | entertainment
Expected | Observed:  | 
Expected | Observed:  | 
Expected | Observed:  | 
Expected | Observed:  | 
Expected | Observed:  | technology


In [34]:
llm_accuracy, llm_avg_time

(0.7, 0.4641483187675476)

In [35]:
router_accuracy, router_avg_time = test_classifier(multi_topic_router, test_data, is_router=True)

Expected | Observed: technology | 
Expected | Observed: technology | 
Expected | Observed: technology | 
Expected | Observed: technology | 
Expected | Observed: technology | technology
Expected | Observed: sports | sports
Expected | Observed: sports | 
Expected | Observed: sports | sports
Expected | Observed: sports | 
Expected | Observed: sports | sports
Expected | Observed: entertainment | 
Expected | Observed: entertainment | entertainment
Expected | Observed: entertainment | 
Expected | Observed: entertainment | entertainment
Expected | Observed: entertainment | entertainment
Expected | Observed:  | 
Expected | Observed:  | 
Expected | Observed:  | 
Expected | Observed:  | 
Expected | Observed:  | 


In [36]:
router_accuracy, router_avg_time

(0.6, 0.02554934024810791)

## Analysis

The following outputs illustrate the tradeoffs with LLMs vs. using the router. As most likely expected the LLM outperforms in terms of accuracy however the router far outperforms the LLM in terms of average latency not to mention the costs incurred with each invocation.